In [2]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [3]:
# 색상 코드를 RGB 소수 값으로 변환
def hex_to_rgb_percent(hex):
    hex = hex.lstrip('#')
    return tuple(int(hex[i:i+2], 16)/255.0 for i in (0, 2, 4))

In [4]:
def get_sheet_id_by_name(service, spreadsheet_id, sheet_name):
    spreadsheet_paste = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheets_paste = spreadsheet_paste.get('sheets', '')
    sheet_order_id = None
    for sheet in sheets_paste:
        if sheet.get('properties', {}).get('title', '') == sheet_name:
            sheet_order_id = sheet.get('properties', {}).get('sheetId', '')
    
    return sheet_order_id

In [5]:
def get_sheet_titles(service, spreadsheet_id):
    # Google Drive API를 사용하여 파일의 메타데이터를 가져옵니다.
    file = service.files().get(fileId=spreadsheet_id, fields='name').execute()
    # 파일 이름(제목)을 반환합니다.
    return file.get('name')

In [6]:
# 특정 셀에 수식을 적용하는 함수
def apply_formula_to_cell_sheet(service, spreadsheet_id, cell_range):
    formula = """={QUERY(A:J, "select E, A, B, C, D, F, H, G where I = 'checked'");{"", "", "", "", "", "", "", ""};{"", "", "", "", "", "", "", ""}; {"", "", "", "", "", "", "", ""};{"도매업체 국내과일만 보기.", "", "", "", "", "", "", ""};{"상품분류", "공급업체명", "상품명", "최종 공급금액 (택배비 포함)", "택배사", "원산지, 브랜드", "출고 마감 일정", "링크"};QUERY(A:J, "select E, A, B, C, D, F, H, G where J = '도매업체, 국내과일'");{"", "", "", "", "", "", "", ""}; {"", "", "", "", "", "", "", ""};{"", "", "", "", "", "", "", ""}; {"농부 국내과일만 보기.", "", "", "", "", "", "", ""};{"상품분류", "공급업체명", "상품명", "최종 공급금액 (택배비 포함)", "택배사", "원산지, 브랜드", "출고 마감 일정", "링크"};QUERY(A:J, "select E, A, B, C, D, F, H, G where J = '농부, 국내과일'");{"", "", "", "", "", "", "", ""}; {"", "", "", "", "", "", "", ""};{"", "", "", "", "", "", "", ""}; {"해외과일만 보기.", "", "", "", "", "", "", ""};{"상품분류", "공급업체명", "상품명", "최종 공급금액 (택배비 포함)", "택배사", "원산지, 브랜드", "출고 마감 일정", "링크"};QUERY(A:J, "select E, A, B, C, D, F, H, G where J = '도매업체, 해외과일'");{"", "", "", "", "", "", "", ""}; {"", "", "", "", "", "", "", ""};{"", "", "", "", "", "", "", ""}; {"농산물만 보기.", "", "", "", "", "", "", ""};{"상품분류", "공급업체명", "상품명", "최종 공급금액 (택배비 포함)", "택배사", "원산지, 브랜드", "출고 마감 일정", "링크"};QUERY(A:J, "select E, A, B, C, D, F, H, G where J = '농산물'");{"", "", "", "", "", "", "", ""}; {"", "", "", "", "", "", "", ""};{"", "", "", "", "", "", "", ""}; {"축산물만 보기.", "", "", "", "", "", "", ""};{"상품분류", "공급업체명", "상품명", "최종 공급금액 (택배비 포함)", "택배사", "원산지, 브랜드", "출고 마감 일정", "링크"};QUERY(A:J, "select E, A, B, C, D, F, H, G where J = '도매업체, 축산물'")}"""
    # A2 셀에 적용할 수식
    value_range_body = {
        'values': [[formula]]  # 적용할 수식
    }
    
    request = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range=cell_range,
        valueInputOption='USER_ENTERED',
        body=value_range_body
    )
    request.execute()

In [7]:
def get_values_from_range(service, spreadsheet_id, range_name):
    # Google Sheets API를 사용하여 특정 범위의 값 가져오기
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    # 값이 있는지 확인 후 반환
    values = result.get('values', [])

    return values

In [8]:
def origin_sheet_copy_and_paste_in_specific_range(service, spreadsheet_id_copy, spreadsheet_id_paste, sheet_order_id, ranges_to_copy):
    # A 스프레드시트의 데이터 및 서식 정보 가져오기
    response = service.spreadsheets().get(
        spreadsheetId=spreadsheet_id_copy,
        ranges=[ranges_to_copy],
        fields='sheets(data((rowData)))'
    ).execute()

    # 읽은 데이터 및 서식 정보
    rows_data = response['sheets'][0]['data'][0]['rowData']

    # B 스프레드시트에 데이터 및 서식 적용
    update_requests = []
    start_column_index = 11  # N열의 시작 인덱스 (0부터 세므로 N열은 13번째 인덱스)
    for row_index, row in enumerate(rows_data):
        for col_index, cell in enumerate(row.get('values', [])):
            value = cell.get('userEnteredValue', {})
            format = cell.get('userEnteredFormat', {})

            # 데이터 적용 요청 추가
            update_requests.append({
                "updateCells": {
                    "rows": [
                        {
                            "values": [
                                {
                                    "userEnteredValue": value,
                                    "userEnteredFormat": format
                                }
                            ]
                        }
                    ],
                    "fields": "userEnteredValue,userEnteredFormat",
                    "range": {
                        "sheetId": sheet_order_id,
                        "startRowIndex": row_index,
                        "endRowIndex": row_index + 1,
                         "startColumnIndex": start_column_index + col_index,
                        "endColumnIndex": start_column_index + col_index + 1
                    }
                }
            })

    # 배치 업데이트로 모든 변경 적용
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id_paste,
        body={"requests": update_requests}
    ).execute()

    print('Data and format copied successfully:', response)

In [9]:
def origin_sheet_copy_and_paste_format_only(service, spreadsheet_id_copy, spreadsheet_id_paste, sheet_order_id, ranges_to_copy):
    # A 스프레드시트의 서식 정보만 가져오기
    response = service.spreadsheets().get(
        spreadsheetId=spreadsheet_id_copy,
        ranges=[ranges_to_copy],
        fields='sheets(data(rowData(values(userEnteredFormat))))'
    ).execute()

    # 읽은 서식 정보
    rows_data = response['sheets'][0]['data'][0]['rowData']

    # B 스프레드시트에 서식만 적용
    update_requests = []
    start_column_index = 0  # L열의 시작 인덱스 (0부터 세므로 L열은 11번째 인덱스)
    for row_index, row in enumerate(rows_data):
        for col_index, cell in enumerate(row.get('values', [])):
            format = cell.get('userEnteredFormat', {})

            # 서식 적용 요청 추가
            update_requests.append({
                "updateCells": {
                    "rows": [
                        {
                            "values": [
                                {
                                    "userEnteredFormat": format
                                }
                            ]
                        }
                    ],
                    "fields": "userEnteredFormat",
                    "range": {
                        "sheetId": sheet_order_id,
                        "startRowIndex": row_index,
                        "endRowIndex": row_index + 1,
                        "startColumnIndex": start_column_index + col_index,
                        "endColumnIndex": start_column_index + col_index + 1
                    }
                }
            })

    # 배치 업데이트로 모든 변경 적용
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id_paste,
        body={"requests": update_requests}
    ).execute()

    print('Format copied successfully:', response)


In [10]:
def origin_sheet_copy_and_paste_with_merge(service, spreadsheet_id_copy, spreadsheet_id_paste, sheet_order_id, ranges_to_copy):
    # A 스프레드시트의 서식 및 병합 정보 가져오기
    response = service.spreadsheets().get(
        spreadsheetId=spreadsheet_id_copy,
        ranges=[ranges_to_copy],
        fields='sheets(data(rowData(values(userEnteredFormat))),merges)'
    ).execute()

    # 읽은 서식 정보
    rows_data = response['sheets'][0]['data'][0]['rowData']
    merges = response['sheets'][0]['merges']  # 병합 정보

    # B 스프레드시트에 서식만 적용
    update_requests = []
    start_column_index = 0  # 시작 인덱스 수정

    # 서식 적용
    for row_index, row in enumerate(rows_data):
        for col_index, cell in enumerate(row.get('values', [])):
            format = cell.get('userEnteredFormat', {})
            update_requests.append({
                "updateCells": {
                    "rows": [{
                        "values": [{
                            "userEnteredFormat": format
                        }]
                    }],
                    "fields": "userEnteredFormat",
                    "range": {
                        "sheetId": sheet_order_id,
                        "startRowIndex": row_index,
                        "endRowIndex": row_index + 1,
                        "startColumnIndex": start_column_index + col_index,
                        "endColumnIndex": start_column_index + col_index + 1
                    }
                }
            })

    # 병합 적용
    for merge in merges:
        update_requests.append({
            "mergeCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": merge['startRowIndex'],
                    "endRowIndex": merge['endRowIndex'],
                    "startColumnIndex": merge['startColumnIndex'],
                    "endColumnIndex": merge['endColumnIndex'],
                },
                "mergeType": "MERGE_ALL"
            }
        })

    # 배치 업데이트로 모든 변경 적용
    return update_requests
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id_paste,
        body={"requests": update_requests}
    ).execute()

    print('Format and merges copied successfully:', response)

In [11]:
def column_row_width_height_copy_and_paste_in_specific_range(service, spreadsheet_id_copy, spreadsheet_id_paste, sheet_order_id, ranges_to_copy):
    # 전체 시트에서 열과 행의 메타데이터 가져오기
    columns_info = service.spreadsheets().get(
        spreadsheetId=spreadsheet_id_copy,
        ranges=ranges_to_copy,
        fields='sheets(data(columnMetadata(pixelSize)))'
    ).execute()

    # 전체 시트에서 행의 메타데이터 가져오기
    rows_info = service.spreadsheets().get(
        spreadsheetId=spreadsheet_id_copy,
        ranges=ranges_to_copy,
        fields='sheets(data(rowMetadata(pixelSize)))'
    ).execute()


    # 열 크기 정보 업데이트 요청
    column_update_requests = []
    # for i, col in enumerate(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK']):
    for i, col in enumerate(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']):
        pixel_size = columns_info['sheets'][0]['data'][0]['columnMetadata'][i]['pixelSize']
        column_update_requests.append({
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_order_id,
                    "dimension": "COLUMNS",
                    "startIndex": ord(col) - ord('A'),  # 열 문자를 인덱스로 변환
                    "endIndex": ord(col) - ord('A') + 1
                },
                "properties": {
                    "pixelSize": pixel_size
                },
                "fields": "pixelSize"
            }
        })

    # 행 크기 정보 업데이트 요청
    row_update_requests = []
    for i in range(0, 27):  # 1~12행 (12행 포함 안되는 경우, i+1 조정 필요)
        pixel_size = rows_info['sheets'][0]['data'][0]['rowMetadata'][i]['pixelSize']
        row_update_requests.append({
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_order_id,
                    "dimension": "ROWS",
                    "startIndex": i,
                    "endIndex": i + 1
                },
                "properties": {
                    "pixelSize": pixel_size
                },
                "fields": "pixelSize"
            }
        })

    # 모든 업데이트 요청을 배치로 실행
    # update_requests = column_update_requests + row_update_requests
    update_requests = column_update_requests + row_update_requests
    return update_requests
    # response = service.spreadsheets().batchUpdate(
    #     spreadsheetId=spreadsheet_id_paste,
    #     body={"requests": update_requests}
    # ).execute()
    # print('Dimensions updated successfully:', response)

In [13]:
def column_to_index(column):
    """엑셀 열 이름(J, O, X 등)을 0부터 시작하는 인덱스로 변환"""
    index = 0
    for char in column:
        index = index * 26 + (ord(char.upper()) - ord('A') + 1)
    return index - 1

def auto_resize_columns(service, spreadsheet_id, sheet_id):
    columns = ['J', 'O', 'X', 'AE', 'AL', 'AS', 'AZ', 'BG', 'BN', 'BU']

    # 각 열 이름에 대해 요청 생성
    requests = []
    for col in columns:
        index = column_to_index(col)
        requests.append({
            "autoResizeDimensions": {
                "dimensions": {
                    "sheetId": sheet_id,
                    "dimension": "COLUMNS",
                    "startIndex": index,
                    "endIndex": index + 1
                }
            }
        })

    
    # return requests
    # API 요청 실행
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={"requests": requests}
    ).execute()

    return response

In [14]:
def fixed_view(service, spreadsheet_id, sheet_order_id):
    # 고정할 행과 열의 수 설정
    frozen_row_count = 9  # 상단 1행부터 10행까지 고정

    # 요청 보내기
    request = {
        "updateSheetProperties": {
            "properties": {
                "sheetId": sheet_order_id,  # 첫 번째 시트
                "gridProperties": {
                    "frozenRowCount": frozen_row_count,
                }
            },
            "fields": "gridProperties.frozenRowCount,gridProperties.frozenColumnCount"
        }
    }

    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={"requests": [request]}
    ).execute()

    print("fixed_view", response)

In [15]:
def hide_columns(service, spreadsheet_id, sheet_order_id):
    # 숨길 열들의 시작과 끝 인덱스 (I, N, O, T, U, Z, AA, AF, AG, AL, AM)
    columns_to_hide = [8, 13, 14, 19, 20, 25, 26, 31, 32, 37, 38, 43, 44, 49, 50, 55, 56, 61, 62]

    # 각 열에 대해 숨기는 요청 생성
    requests = [{
        "updateDimensionProperties": {
            "range": {
                "sheetId": sheet_order_id,  # 시트 ID
                "dimension": "COLUMNS",  # 열 변경
                "startIndex": col_index,  # 시작 인덱스
                "endIndex": col_index + 1  # 종료 인덱스 (startIndex와 같은 열을 숨김)
            },
            "properties": {
                "hiddenByUser": True
            },
            "fields": "hiddenByUser"
        }
    } for col_index in columns_to_hide]

    # 요청 실행

    return requests
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={"requests": requests}
    ).execute()

    print(response)


In [16]:
def apply_formula_to_muti_cells_number(service, spreadsheet_id, range_name):
    formulas = []
    for i in range(12, 1011):  # 11행부터 1010행까지
        formula = f'=IF(ISBLANK(U{i}), "", IF(LEN(U{i}) >= 10, HYPERLINK(U{i}, "사진보기 ←클릭"), IF(AND(LEN(U{i}) >= 1, LEN(U{i}) <= 5), "상품상세정보 및 사진링크", "")))'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()


In [17]:
def create_new_sheet(service, spreadsheet_id, title):
    # 새 시트 추가 요청
    request_body = {
        'requests': [
            {
                'addSheet': {
                    'properties': {
                        'title': title,
                        'gridProperties': {
                            'rowCount': 1030,
                            'columnCount': 80
                        }
                    }
                }
            }
        ]
    }

    # 요청 실행
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body=request_body
    ).execute()

    print('Sheet created successfully:', response)
    return response


In [ ]:
# def all_set_for_speed()

In [18]:
def importrangeAllowAccessAutomation(get_spreadsheet_id, give_spreadsheet_id, creds):
    url = f'https://docs.google.com/spreadsheets/d/{get_spreadsheet_id}/externaldata/addimportrangepermissions?donorDocId={give_spreadsheet_id}&includes_info_params=true&cros_files=false'
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {creds.token}',
    }
    # 데이터를 JSON 형태로 변환하고, POST 요청을 보냅니다.
    response = requests.post(url, headers=headers)
    # 응답 확인
    if 'application/json' in response.headers.get('Content-Type', ''):
        try:
            data = response.json()
            print(data)
        except ValueError:
            print("JSON 디코딩 실패", response.text)
    else:
        print("응답이 JSON 형식이 아닙니다:", response.text)


In [19]:
import time
def importrange_seller_to_imp(service_sheets, paste_spreadsheet_id, copy_spreadsheet_id, creds):
    range_name = '공지사항!A1'
    formula = f'=IMPORTRANGE(\"{copy_spreadsheet_id}\","\'공지사항모아보기\'!$A$1:$BZ")'  # A2 셀에 적용할 수식
    value_range_body = {
        "range": range_name,
        "majorDimension": "ROWS",
        "values": [[formula]]
    }
    request = service_sheets.spreadsheets().values().update(
    spreadsheetId=paste_spreadsheet_id, range=range_name,
    valueInputOption="USER_ENTERED", body=value_range_body)
    response = request.execute()

    # print(f'Updated {sheet_title}!A2 with formula: {formula}')

    importrangeAllowAccessAutomation(paste_spreadsheet_id, copy_spreadsheet_id, creds)


In [20]:
import time
def insert_one_cell(service_sheets, paste_spreadsheet_id):
    range_name = '공지사항!CA3'
    formula = f'=HYPERLINK(\"https://docs.google.com/spreadsheets/d/1p0WM4X2JztS1LfGKVdUXd3GTvBJJGYPNO0ya8ihfns4/edit#gid=1594213233\",\"지난공지사항 더 보러가기\")'  # A2 셀에 적용할 수식
    value_range_body = {
        "range": range_name,
        "majorDimension": "ROWS",
        "values": [[formula]]
    }
    request = service_sheets.spreadsheets().values().update(
    spreadsheetId=paste_spreadsheet_id, range=range_name,
    valueInputOption="USER_ENTERED", body=value_range_body)
    response = request.execute()

In [21]:
import time

def change_format_point(service_sheets, spreadsheet_copy_id, spreadsheet_paste_id, creds):
    
    # # 목적지 스프레드시트 및 해당 시트 ID 찾기    
    # sheet_order_id_공지사항 = get_sheet_id_by_name(service_sheets, spreadsheet_paste_id, "공지사항")
    sheet_order_id_공지사항 = create_new_sheet(service_sheets, spreadsheet_paste_id, "공지사항")["replies"][0]["addSheet"]["properties"]["sheetId"]
    # time.sleep(0.5)
    # sheet_order_id_공지사항 = get_sheet_id_by_name(service_sheets, spreadsheet_paste_id, "공지사항")
    time.sleep(0.5)
    a = origin_sheet_copy_and_paste_with_merge(service_sheets, spreadsheet_copy_id, spreadsheet_paste_id, sheet_order_id_공지사항, "공지사항모아보기!A1:BZ")
    # apply_formula_to_cell_sheet(service_sheets, spreadsheet_paste_id, "상품목록!N13:N13")
    # time.sleep(0.5)
    # add_conditional_format_rule(service_sheets, spreadsheet_paste_id, sheet_order_id_상품목록)
    # time.sleep(0.5)
    # apply_formula_to_muti_cells_number(service_sheets, spreadsheet_paste_id, "상품목록!V12:V1010")
    # time.sleep(0.5)
    b = column_row_width_height_copy_and_paste_in_specific_range(service_sheets, spreadsheet_copy_id, spreadsheet_paste_id, sheet_order_id_공지사항, "공지사항모아보기!A1:BZ")

    # c = auto_resize_columns(service_sheets, spreadsheet_paste_id, sheet_order_id_공지사항)
    # time.sleep(0.5)
    # fixed_view(service_sheets, spreadsheet_paste_id, sheet_order_id_상품목록)
    # time.sleep(0.5)
    # c = hide_columns(service_sheets, spreadsheet_paste_id, sheet_order_id_공지사항)

    response = service_sheets.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_paste_id,
        body={"requests": a + b}
    ).execute()

    importrange_seller_to_imp(service_sheets, spreadsheet_paste_id, spreadsheet_copy_id, creds)
    insert_one_cell(service_sheets, spreadsheet_paste_id)

spreadsheet_copy_id = "1p0WM4X2JztS1LfGKVdUXd3GTvBJJGYPNO0ya8ihfns4"
spreadsheet_paste_id = "1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE"



# if __name__ == "__main__":
#     creds = oauthByWeb()
#     service_sheets = build('sheets', 'v4', credentials=creds)
#     service_drive = build("drive", "v3", credentials=creds) 

#     change_format_point(service_sheets, spreadsheet_copy_id, spreadsheet_paste_id, creds)
    

In [23]:
user_sheet_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
spreadsheet_copy_id = "1p0WM4X2JztS1LfGKVdUXd3GTvBJJGYPNO0ya8ihfns4"
import time


if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds) 
    range_name = "셀러컨택포인트!A240:T464"
    # range_name = "셀러컨택포인트!A2:T4"
    # range_name = "셀러컨택포인트!A265:T267"
    user_data_list = get_values_from_range(service_sheets, user_sheet_id, range_name)
    print(user_data_list)

     
    for user_data in user_data_list:
        time.sleep(2)

        if user_data[10]:
            print("user_data", user_data[1])
            user_id = user_data[10]
            change_format_point(service_sheets, spreadsheet_copy_id, user_id, creds)

            print(user_data[1])
        






[['', '258', '엘리샵 710', '258_엘리샵 710', '', '', '', '', '', '1025639544', '164jZ_ebAh_SeJ_MNnH4la8ZRSOUzIw8qlNnaX-eg9eI', 'https://docs.google.com/spreadsheets/d/164jZ_ebAh_SeJ_MNnH4la8ZRSOUzIw8qlNnaX-eg9eI', 'eliana080@gmail.com', '', '', '', '', '', '', '1'], ['', '259', '리브드릴리', '259_리브드릴리', '', '', '', '', '', '1057802238', '1SB_p-YSE_Dsh17jBY3aWVP14dUJt8vdsYV33niPe8zM', 'https://docs.google.com/spreadsheets/d/1SB_p-YSE_Dsh17jBY3aWVP14dUJt8vdsYV33niPe8zM', 'canadareadingbears.lily@gmail.com', '', '', '', '', '', '', '1'], ['', '260', '포동포동 농수산물', '260_포동포동 농수산물', '', '', '', '', '', '1063312935', '1xD9hJnw7s8BNMuFGpC7uiO2YJsjNPRYQoObvNsxo4sE', 'https://docs.google.com/spreadsheets/d/1xD9hJnw7s8BNMuFGpC7uiO2YJsjNPRYQoObvNsxo4sE', 'songh4465@gmail.com', '송현우', '', '', '', '', '', '1'], ['', '261', '젤마마켓', '261_젤마마켓', '', '', '', '', '', '1041590971', '1-26PbFTHeDXxaawArrnjQ5-S3OGSQc6MlCy2MpeSiNE', 'https://docs.google.com/spreadsheets/d/1-26PbFTHeDXxaawArrnjQ5-S3OGSQc6MlCy2MpeSiNE', '

HttpError: <HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1vVrVXYE4bNOSGeOXfSRcXTjnlqFUzaarJ1t91GNhk5k:batchUpdate?alt=json returned "Invalid requests[0].addSheet: 이름이 ‘공지사항’인 시트가 이미 있습니다. 다른 이름을 입력해 주세요.". Details: "Invalid requests[0].addSheet: 이름이 ‘공지사항’인 시트가 이미 있습니다. 다른 이름을 입력해 주세요.">

In [3]:
def apply_formula_to_muti_cells(service, spreadsheet_id, range_name, phone_number):
    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", \"{phone_number}\")'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()


In [1]:
def apply_formula_to_muti_cells_chords(service, spreadsheet_id, range_name, sheet_title):
    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"",{sheet_title})'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()


In [37]:
def apply_formula_to_muti_cells_number(service, spreadsheet_id, range_name):
    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", 1)'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()


In [36]:
# 특정 셀에 수식을 적용하는 함수
def apply_formula_to_cell_sheet(service, spreadsheet_id, cell_range):
    formula = f'=sum(K11:K950)'  # A2 셀에 적용할 수식
    range_name = cell_range  # 지정된 시트의 특정 셀 범위
    value_range_body = {
        'values': [[formula]]  # 적용할 수식
    }
    
    request = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range="발주서!K8",
        valueInputOption='USER_ENTERED',
        body=value_range_body
    )
    request.execute()

In [16]:
def insert_title_text_in_cell(service, spreadsheet_id, sheet_title, sheet_order_id):
    
    cell_range_e3 = 'E2'  # 수식을 적용할 셀 범위
    range_name_e3 = f"{sheet_title}!{cell_range_e3}"  # 지정된 시트의 특정 셀 범위    

    text_e3 = f"2. 보낼 금액을 확인 후 입금합니다. "

    color_hex = "#2F55CB"

    
    blue = {"red": int(color_hex[1:3], 16) / 255.0, "green": int(color_hex[3:5], 16) / 255.0, "blue": int(color_hex[5:7], 16) / 255.0}  # 검은색
    black =  {"red": 0.0, "green": 0.0, "blue": 0.0}  # 검은색
    font_size_14 = 14  # 적용할 글자 크기
    font_size_12 = 12  # 적용할 글자 크기
    font_size_25 = 20  # 적용할 글자 크기
    bold = True  # 볼드 여부

    text_format_e3 = {
        "foregroundColor": black,
        "fontSize": font_size_12,
        "bold": bold
    }

    request_3 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_e3}, "userEnteredFormat": {"textFormat": text_format_e3}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 1, 
                'startColumnIndex': 4, 
                'endRowIndex':2,
                'endColumnIndex':5
            }
        }
    }

    response_3 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_3]}
    ).execute()


In [1]:
def parse_cell_range(cell_address):
    import re

    # 정규식을 사용하여 셀 주소에서 시트 이름, 열, 행을 분리합니다.
    match = re.match(r"([a-zA-Z]+)!([A-Z]+)(\d+)", cell_address)
    if not match:
        raise ValueError("Invalid cell address format")

    sheet_name, col, row = match.groups()
    row_index = int(row) - 1  # 1-based index를 0-based index로 변환
    col_index = column_to_index(col) - 1  # 열 이름을 0-based index로 변환

    # 시작 및 끝 인덱스는 하나의 셀이므로 동일합니다.
    return {
        "sheetName": sheet_name,
        "startRowIndex": row_index,
        "endRowIndex": row_index + 1,
        "startColumnIndex": col_index,
        "endColumnIndex": col_index + 1
    }

def column_to_index(column):
    """ 열 이름 (예: 'A', 'B', ..., 'Z', 'AA', etc.)을 0-based index로 변환합니다. """
    index = 0
    for char in column:
        index = index * 26 + (ord(char) - ord('A') + 1)
    return index


In [47]:
# 색상 코드를 RGB 소수 값으로 변환
def hex_to_rgb_percent(hex):
    hex = hex.lstrip('#')
    return tuple(int(hex[i:i+2], 16)/255.0 for i in (0, 2, 4))

def background_color_change(service, spreadsheet_id, sheetId):
    # 배경색 설정
    background_color = hex_to_rgb_percent("7AD592")
    requests = [
    # A1 셀의 배경색 설정 및 텍스트 삭제
    {
        'repeatCell': {
            'range': {
                'sheetId': sheetId,
                'startRowIndex': 8,
                'endRowIndex': 9,
                'startColumnIndex': 8,
                'endColumnIndex': 9
            },
            'cell': {
                'userEnteredFormat': {
                    'backgroundColor': {
                        'red': background_color[0],
                        'green': background_color[1],
                        'blue': background_color[2]
                    }
                },
                'userEnteredValue': {}  # 텍스트를 삭제
            },
            'fields': 'userEnteredFormat.backgroundColor, userEnteredValue'
        }
    },
    # B1 셀의 배경색 설정
    {
        'repeatCell': {
            'range': {
                'sheetId': sheetId,
                'startRowIndex': 9,
                'endRowIndex': 10,
                'startColumnIndex': 8,
                'endColumnIndex': 9
            },
            'cell': {
                'userEnteredFormat': {
                    'backgroundColor': {
                        'red': background_color[0],
                        'green': background_color[1],
                        'blue': background_color[2]
                    }
                }
            },
            'fields': 'userEnteredFormat.backgroundColor'
        }
    }
]


    # 요청 실행
    body = {
        'requests': requests
    }
    response = service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=body).execute()

    print('Response:', response)

In [6]:
user_sheet_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
spreadsheet_copy_id = "1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE"
import time


if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds) 
    # range_name = "셀러컨택포인트!A305:T330"
    range_name = "셀러컨택포인트!A331:T332"
    user_data_list = get_values_from_range(service_sheets, user_sheet_id, range_name)
    print(user_data_list)

    origin_sheet_id = "1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE"

    origin_phone_number = "01051319792"

    # apply_formula_to_muti_cells(service_sheets, origin_sheet_id, "발주서!I11:I", origin_phone_number)

     
    for user_data in user_data_list:
        time.sleep(1)

        if user_data[10] == "":
            continue

        if user_data[1] == "24":
            continue

        phone_number = "01051319792"
        if user_data[9]:
            phone_number = user_data[9]
            

        print("user_data", user_data[1])
        user_id = user_data[10]
        user_insert_name = user_data[13]
        # sheet_title = get_sheet_titles(service_drive, user_id)
        sheet_title = user_data[1]

        insert_name = sheet_title

        if user_insert_name:
            insert_name = user_insert_name

        
        apply_formula_to_muti_cells_chords(service_sheets, user_id, "발주서!J11:J", sheet_title)
        # apply_formula_to_muti_cells_number(service_sheets, user_id, "발주서!D11:D")
        # sheet_order_id = get_sheet_id_by_name(service_sheets, user_id, "발주서")
        # background_color_change(service_sheets, user_id, sheet_order_id)
        



[['', '349', '해피상회', '', '', '', '', '', '', '1041677881', '1WqF0xezvhkCro5f7aIn3yhmmD8x5fve4sFC18qa8yn4', 'https://docs.google.com/spreadsheets/d/1WqF0xezvhkCro5f7aIn3yhmmD8x5fve4sFC18qa8yn4', 'ymee788@gmail.com', '', '', '', '', '', '', '1'], ['', '350', '소담유통', '', '', '', '', '', '', '1076890334', '19ncCK2cIQG4Yu3j4AjhzHbyRHadPnU5LBeJ1Pu-q9wg', 'https://docs.google.com/spreadsheets/d/19ncCK2cIQG4Yu3j4AjhzHbyRHadPnU5LBeJ1Pu-q9wg', 'moonseungsu123@gmail.com', '', '', '', '', '', '', '1']]
user_data 349
user_data 350


In [ ]:
user_id = "1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE"


if __name__ == "__main__":
    creds = oauthByWeb()
    phone_number = "01051319792"
    service_sheets = build('sheets', 'v4', credentials=creds)
    apply_formula_to_muti_cells(service_sheets, user_id, "발주서!I11:I", phone_number)

In [50]:
def aws_apply_formula_to_muti_cells_phone_number(service, spreadsheet_id, range_name, phone_number):
    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", \"{phone_number}\")'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()


In [51]:
def aws_insert_title_text_in_cell(service, spreadsheet_id, sheet_order_id, insert_name):
    text_c2 = f"입금 시 입금자명으로 \"{insert_name}\"을 입력해주세요."
    text_e3 = f"3. 입금자명을 \"{insert_name}\"으로 입력합니다."
    text_l8 = f"{insert_name}"

    color_hex = "#2F55CB"

    
    blue = {"red": int(color_hex[1:3], 16) / 255.0, "green": int(color_hex[3:5], 16) / 255.0, "blue": int(color_hex[5:7], 16) / 255.0}  # 검은색
    black =  {"red": 0.0, "green": 0.0, "blue": 0.0}  # 검은색
    font_size_14 = 14  # 적용할 글자 크기
    font_size_12 = 12  # 적용할 글자 크기
    font_size_25 = 20  # 적용할 글자 크기
    bold = True  # 볼드 여부

    text_format_c2 = {
        "foregroundColor": blue,
        "fontSize": font_size_14,
        "bold": bold
    }

    text_format_e3 = {
        "foregroundColor": black,
        "fontSize": font_size_12,
        "bold": bold
    }

    text_format_l8 = {
        "foregroundColor": black,
        "fontSize": font_size_25,
        "bold": bold
    }

    request_2 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_c2}, "userEnteredFormat": {"textFormat": text_format_c2}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 1, 
                'startColumnIndex': 2, 
                'endRowIndex':2,
                'endColumnIndex':3
            }
        }
    }

    request_3 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_e3}, "userEnteredFormat": {"textFormat": text_format_e3}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 2, 
                'startColumnIndex': 4, 
                'endRowIndex':3,
                'endColumnIndex':5
            }
        }
    }


    request_4 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_l8}, "userEnteredFormat": {"textFormat": text_format_l8}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 7, 
                'startColumnIndex': 11, 
                'endRowIndex':8,
                'endColumnIndex':12
            }
        }
    }    

    response_2 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_2]}
    ).execute()

    response_3 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_3]}
    ).execute()

    response_4 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_4]}
    ).execute()

In [53]:
def aws_get_sheet_id_by_name(service, spreadsheet_id, sheet_name):
    spreadsheet_paste = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheets_paste = spreadsheet_paste.get('sheets', '')
    sheet_order_id = None
    for sheet in sheets_paste:
        if sheet.get('properties', {}).get('title', '') == sheet_name:
            sheet_order_id = sheet.get('properties', {}).get('sheetId', '')
    
    return sheet_order_id

In [54]:
def lambda_handler(event, context): 
    creds = oauth()
    service = build('sheets', 'v4', credentials=creds)
    spreadsheet_id = event["spreadsheet_id"]
    insert_name = event["insert_name"]
    phone_number = event["phone_number"]
    sheet_order_id = aws_get_sheet_id_by_name(service, spreadsheet_id, "발주서")

    try: 
        if phone_number:
            aws_apply_formula_to_muti_cells_phone_number(service, spreadsheet_id, "발주서!I11:I", phone_number)
        if insert_name:
            aws_insert_title_text_in_cell(service, spreadsheet_id, sheet_order_id, insert_name)
    except HttpError as error:
        # HttpError에서 응답 코드와 메시지를 추출합니다.
        error_code = error.resp.status
        try:
            # 오류 응답 본문을 JSON 객체로 파싱합니다.
            error_details = json.loads(error.content.decode())
            error_message = error_details.get('error', {}).get('message', 'Unknown error')
        except json.JSONDecodeError:
            # 오류 응답 본문이 JSON이 아닌 경우, 일반 텍스트로 처리합니다.
            error_message = error.content.decode()

        print(f"An error occurred: {error}")
        return {
            'statusCode': error_code,  # HTTP 상태 코드를 응답에 설정합니다.
            'body': json.dumps({"error": error_message})  # 오류 메시지를 JSON 응답에 포함시킵니다.
        }

    return {
        'statusCode': 200,
        'body': json.dumps({"result": "Success", "message": "Formula applied successfully"})
    }

In [56]:
if __name__ == "__main__":

    lambda_handler({
        "spreadsheet_id": "1RAnWFmNvlv2VUk_biNbpUyhasr6o2Gt5HXZEk216sDs",
        "insert_name": "338_백민기",
        "phone_number":"01052371951"
    }, {})
